# Isolation Forest

![basic isolation](img/basic-isolation-example.png)

## 1. Introdução

Isolation Forest é um algoritmo eficiente de deteção de dados extremos, especialmente em datasets com alta dimensionalidade.
O algoritmo constroi uma random forest em cada árvore de decisão cresce aleatoriamente. Em cada nó ele seleciona uma feature aleatoriamente, depois seleciona um threshold aleatório para dividir o dataset em dois. O algoritmo divide o dataset gradualmente até que todas as instâncias estejam isoladas das outras instâncias. Outliers normalmente estão longe de outras instâncias, logo esses datapoints tendem a ser isolados em menos etapas que datapoints "normais". 

## 2. Dataset

A GENTE PODE USAR UM DATASET QUALQUER OU CRIAR UM E GERAR ALGUNS OUTLIERS MATEMATICAMENTE NELES (A GENTE PODE USAR ISSO PRA MOSTRAR UM EXEMPLO SIMPLIFICADO E DEPOIS ABORDAR UM DATASET GRANDE). VOU DEIXAR O ARTIFICIAL AQUI EMBAIXO.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

uniformSkewed = np.random.rand(10000) * 100 - 40
high_outliers = np.random.rand(38) * 50 + 100
low_outliers = np.random.rand(10) * -50 - 100

data = np.concatenate((uniformSkewed, high_outliers, low_outliers))
plt.boxplot(data)
plt.show()

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

n_samples, n_outliers = 120, 40
rng = np.random.RandomState(0)
covariance = np.array([[0.5, -0.1], [0.7, 0.4]])
cluster_1 = 0.4 * rng.randn(n_samples, 2) @ covariance + np.array([2, 2])  # general
cluster_2 = 0.3 * rng.randn(n_samples, 2) + np.array([-2, -2])  # spherical
outliers = rng.uniform(low=-4, high=4, size=(n_outliers, 2))

X = np.concatenate([cluster_1, cluster_2, outliers])
y = np.concatenate(
    [np.ones((2 * n_samples), dtype=int), -np.ones((n_outliers), dtype=int)]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

scatter = plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor="k")
handles, labels = scatter.legend_elements()
plt.axis("square")
plt.legend(handles=handles, labels=["outliers", "inliers"], title="true class")
plt.title("Gaussian inliers with \nuniformly distributed outliers")
plt.show()

In [ ]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples=100, random_state=0)
clf.fit(X_train)

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

disp = DecisionBoundaryDisplay.from_estimator(
    clf,
    X,
    response_method="predict",
    alpha=0.5,
)
disp.ax_.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor="k")
disp.ax_.set_title("Binary decision boundary \nof IsolationForest")
plt.axis("square")
plt.legend(handles=handles, labels=["outliers", "inliers"], title="true class")
plt.show()

In [ ]:
disp = DecisionBoundaryDisplay.from_estimator(
    clf,
    X,
    response_method="decision_function",
    alpha=0.5,
)
disp.ax_.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor="k")
disp.ax_.set_title("Path length decision boundary \nof IsolationForest")
plt.axis("square")
plt.legend(handles=handles, labels=["outliers", "inliers"], title="true class")
plt.colorbar(disp.ax_.collections[1])
plt.show()